In [ ]:
!pip install -U nlp

In [ ]:
!pip install -U datasets

In [ ]:
!pip install --upgrade tensorflow-datasets

In [ ]:
!pip install colorama

In [ ]:
!pip install sounddevice

In [ ]:
!pip install config

In [ ]:
!pip install openai-whisper

In [ ]:
!pip install wavio

In [5]:
!pip install tensorflow

In [3]:
!pip install ffmpeg

In [18]:
!pip install tensorflow

  Using cached tensorflow-2.16.1-cp311-cp311-macosx_12_0_arm64.whl.metadata (4.1 kB)
  Using cached ml_dtypes-0.3.2-cp311-cp311-macosx_10_9_universal2.whl.metadata (20 kB)
  Using cached tensorboard-2.16.2-py3-none-any.whl.metadata (1.6 kB)
  Using cached keras-3.2.0-py3-none-any.whl.metadata (5.6 kB)
Using cached tensorflow-2.16.1-cp311-cp311-macosx_12_0_arm64.whl (227.0 MB)
Using cached keras-3.2.0-py3-none-any.whl (1.1 MB)
Using cached ml_dtypes-0.3.2-cp311-cp311-macosx_10_9_universal2.whl (389 kB)
Using cached tensorboard-2.16.2-py3-none-any.whl (5.5 MB)


In [25]:
!pip install --upgrade tensorflow keras

In [ ]:
# all libs for audio/whisper transcription
import sounddevice as sd # recording compatibility
import wavio as wv # audio file compatibilty
import datetime # to ensure it is live recording by keeping track of time
import config
import os, glob
import whisper
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Embedding, Bidirectional, LSTM, AveragePooling1D, BatchNormalization
import tensorflow.keras as keras
import numpy as np # brings functionality of arrays data structure to python, used a lot in machine learning
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from datasets import load_dataset
import pandas as pd
import time
import random
from colorama import  Back, Style, Fore



# Assuming your dataset is in CSV format with columns 'text' and 'label'
# Adjust the file path accordingly
file_path = 'nlpData/training.csv'

# Load the dataset using pandas
df = pd.read_csv(file_path)

# Display the first few rows of the dataset
print(df.head())

# Access the 'text' and 'label' columns
texts = df['text'].tolist()
labels = df['label'].tolist()

# generic for all emotions
RULER = """
Let's utilize the RULER method to manage this tough emotion you are experiencing. 
First sense it and be mindful of it let it be there. Stop and pause and think of your best self. 
To get to your best self think of ways that help enhance your mood right now.
"""

# sad feedbacks
sad_1 = """
Pause.  Your words may impact others wrongly so please take a pause. 
Is what your saying really going to help? 
Make a list of pros and cons if you further persist with this attitude. 
If there are a lot of cons then please control yourself before impacting your relationships. 
Try engaging with your senses through ice packs, scents, etc. 
or walk away from the situation (if possible) and perform a joyful activity. 
Remember not all content has to be tried out, these are just ideas from your personal emotion regulation therapist.
"""

sad_2 = """
When others may hear what you say they may feel hurt. 
If it is commenting on someone, keep it to yourself so your relationships do not get damaged. 
If you are thinking of ending a relationship do it in a fulfilling manner where the other individual 
does not feel offended by speaking assertively and using “I” statements. 
Remember not all content has to be tried out, these are just ideas from your personal emotion regulation therapist.
"""

sad_3 = """
If you are in a moderate place, ask yourself, “why am I sad?”. 
If not, then work on distractions to distract that emotion and come back to that question of why. 
Accepting and being mindful of the situation you are encountering may help you come up with solutions for a better outcome. 
Remember not all content has to be tried out, these are just ideas from your personal emotion regulation therapist. 
Remember (think of your own positive affirmation as you know yourself the best)
"""

sad_4 = """
Sometimes it is better to reach out to people to share your pain with them. 
Start by using “I” statements and speaking assertively to make it easy to contact with others around you for support. 
Try positive mantras in your head example of a positive mantra:
"I am doing my best everyday" 
The senses are also a great distraction to take your mind off of what you are sad and engage in something soothing for you.
"""

sad_5 = """
Consider doing opposite action. This is when you want to do the opposite of what your thoughts are impulsing you to do. 
But first, check if your emotion fits well to the scenario you are facing.Trust me, this will work easy peasy. 
Think about the pros and cons if you do what you think of doing right now? Will it serve you long term?
If it does not, do not do it.
Another resource is to change your body temperature to distract from something else.
Take a break from the situation and go grab an ice pack or whatever temperature/texture is soothing for you. 
Think of others around you so you can refrain yourself from anything aggressive and come to wise or rational mind.
Remember not all content has to be tried out, these are just ideas from your personal emotion regulation therapist.
"""

sad_6 = """
Engage in pleasant activities to further enhance your mood or be mindful of the emotion that is occurring right now. 
Remember not all content has to be tried out, these are just ideas from your personal emotion regulation therapist.
"""

sad_7 = """
Guided Meditation/self talk:
Now hear yourself saying: Each day, in every way, I get better and better. 
In order to make change I challenge myself. 
When I have a negative thought, I acknowledge that I feel this way and then I release it. When another negative thought arises, I catch it quickly and I say with confidence, I acknowledge that part of me feels this way, but this thought does not help me right now. 
Each day, in every way I get better and better. 
Every single day, I become more aware of all the good things going on in my community, and in the world around me. 
When I feel that darkness is surrounding me, I know that the way to see light is to take 3 deep breaths, 
relaxing my mind and body by doing so. 
In order to make great change, I challenge my thoughts. 
"""

sad_8 = """
These are best for when in grief/distress.
1. Stay in touch with others around you so they know how you are feeling.
2. Exercise is a great way to alleviate yourself from distress but please do so if you have the medical capability. Try participating in healthy activities that you find enjoyable.
3. Keeping up with good nutriiton will enable the nutrients to better help you shift into wise mind (where emotion mind and rational mind are balanced). It is okay to not be achieving your wise mind on your first try.
4. Routines always help as you know what will happen next rather having a new emotion about an uncertain event. Example of a routine can be found here: https://www.calm.com/blog/daily-routine
5. Enjoy and be gracious for all the resources this world has bestowed upon you, it will help you think of the brighter side of the picture.
"""

# list to randomize these (all of them are helpful)
sad_feedbacks = [sad_1, sad_2, sad_3, sad_4, sad_5, sad_6, sad_7, sad_8, RULER]


# fearful feedbacks
fearful_1 = """
Anxiety can be overwhelming. But emohelp.ai is here to help! Our system has noticed you are feeling anxious. 
You should first ground yourself and sit with the anxiety. When sitting with anxiety, start noticing what is around you. 
5 things you can see, 4 things you can touch, 3 things you can hear, 2 things you can smell, and 1 thing you can taste. 
Now to contact the senses rather than noticing them, self-soothe yourself with the senses that appeal to you in a calming manner. 
That could be taking a warm bath, rubbing your hands through velvet, etc. Be aware of not engaging in sensory activities 
that disturb you! Distraction is also key to overcoming anxiety. Distractions help as 
they shift the mind from unwanted thoughts to the thoughts your wise mind desires to be fed by. 
Examples of distractions can be, watching TV, talking to your loved ones, walking your pet (if you have one), 
doing community service, etc. If it is a social situation, think of this experience as an exposure get 
yourself exposed to those fears to cure your anxiety in the long term. Use “I” statements to speak up.
Remember not all content has to be tried out, these are just ideas from your personal emotion regulation therapist.
"""

fearful_2 = """
If it was something that was actually distressing, speaking of rational terms, try using pleasing imagery playing in your mind 
to lift off the disgusting thoughts. However, if it is not a reasonably disgusting moment, exposure may be advised. 
To start off with the exposure of your fear, your values and goals can be used as a motivator to complete the exposure for doing 
what is “distressing”. Remember not all content has to be tried out, these are just ideas from 
your personal emotion regulation therapist.

Here is a video of a pleasant imagery guided meditation:
https://youtu.be/AbckuluEdM0
"""

fearful_3 = """
Take a step back and realize what’s going on. Reframe the anxious thoughts you are having. 
Anxious thoughts can be identified by the thoughts that start bringing up fear in your body through sensations 
or just a feeling you know. Communicate your needs if the anxiety is coming from a third party source. 
To advocate for your needs it could mean redirecting people around you that are causing your anxiety. 
It may be anxious to do so but remember the outcome of advocating for your needs–it will help you overcome your anxiety. 
Try telling the person who is with you what you are scared of by using “I” statements. 
Remember not all content has to be tried out, these are just ideas from your personal emotion regulation therapist.

To have a powerful presence in your converstation of communication your needs, here is a video to help:
https://youtu.be/mq8YmWaybeE?t=166
"""

fearful_4 = """
Envision your favorite place in the world where you feel best. 
Imagine your loved one sitting next to you watching this moment arise.
Now, you fly with your loved one to a place of minimal worry.
"""

fearful_5 = """
Guided Meditation:
Get comfortable. You can lie on your back in bed or on the floor with a pillow under your head and knees.
Or you can sit in a chair with your shoulders, head, and neck supported against the back of the chair.
Breathe in through your nose. Let your belly fill with air.
Breathe out through your nose.
Place one hand on your belly. Place the other hand on your chest.
As you breathe in, feel your belly rise. As you breathe out, feel your belly lower.
The hand on your belly should move more than the one that's on your chest.
Take three more full, deep breaths. Breathe fully into your belly as it rises and falls with your breath.
Whenever I am presented with feelings of stress, 
I know my breath is there to soothe me and guide me into relaxation.
"""

fearful_6 = """
Live by the facts. What is evidence of the fear you hold? Will this matter in 1, 2, 5, 25 years? 
Answering these questions 
may make you realize whether or not this fear is rational. Having that knowledge, it is hoped for your success in 
overcoming this troubling experience. For example, you go into a social gathering and you get the wrong perception 
of people so you are scared for your life that they may embarrass you. Think about the evidence supporting your fear 
and evidence against it. As this is most likely an irrational fear, you may find that there is more evidence against it. 
Since you are finding more of the unsupportive evidence, there is a possibility that it would be easier for you to realize
that people embarrasing you lasts short-term.
"""

fearful_7 = """
This anxiety can be cured with simplest of things. Having a routine to divert your focus may help you conquer your everday goals without that anxiety
being a hurdle. To begin, everyday things like a good night’s sleep, a wholesome meal and a walk are often the best cures for anxiety.
Even it could be having supportive people around you and make a tradition to do an activity with them recurrently.

Here is a guide on how to make the best routine: 
https://www.calm.com/blog/daily-routine
"""

fearful_8 = """
Try to make rewards for everything you accomplish to increase your self-compassion and motivation to face your anxiety or fear. If you know
that for example you might get ice cream after being in a social situation that may help you to engage in a converstation.
"""

fearful_feedbacks = [fearful_1, fearful_2, fearful_3, fearful_4, fearful_5, fearful_6, fearful_7, fearful_8, RULER]

angry_1 = """"
Take a step back and realize what’s going on. Reframe the anxious thoughts you are having. 
angry thoughts can be identified by the thoughts that start bringing up angry heat in your body through sensations 
or just a feeling you know. Communicate your needs if the anger is coming from someone else. 
To advocate for your needs it could mean redirecting people around you that are causing your anger. 
It may be anxious to do so but remember the outcome of advocating for your needs–it will help you overcome your anxiety. 
Try telling the person who is with you who triggered you by using “I” statements. 
Remember not all content has to be tried out, these are just ideas from your personal emotion regulation therapist.
"""

angry_2 = """
Drink water, sit down, think of the positives of the aspect.
Get yourself out of emotion mind and into wise mind by thinking if the 
emotion really fits the facts.
"""

angry_3 = """
Consider doing opposite action. This is when you want to do the opposite of what your thoughts are impulsing you to do. 
But first, check if your emotion fits well to the scenario you are facing.Trust me, this will work easy peasy. 
Think about the pros and cons if you do what you think of doing right now? Will it serve you long term?
If it does not, do not do it.
Another resource is to change your body temperature to distract from something else.
Take a break from the situation and go grab an ice pack or whatever temperature/texture is soothing for you. 
Think about others around you, how they would feel hurt by your actions triggered by your anger. 
Think of them so you can refrain yourself from anything aggressive and come to wise or rational mind.
Remember not all content has to be tried out, these are just ideas from your personal emotion regulation therapist.
"""

angry_4 = """
When others may hear what you say they may feel hurt. 
If it is commenting on someone, keep it to yourself so your relationships do not get damaged. 
If you are thinking of ending a relationship do it in a fulfilling manner where the other individual 
does not feel offended by speaking assertively and using “I” statements. 
Remember not all content has to be tried out, these are just ideas from your personal emotion regulation therapist.

Try this video on anger management: 
https://youtu.be/hxZS50WhhBI
"""

angry_5 = """
It’s time to take a break…. and relax…. to deal with anger in a healthy, productive way.

Anger is a normal and natural emotion, and there is nothing wrong with having feelings – you are human, after all. 
You have the power to decide how to deal with this emotion you are experiencing.

Anger management does not mean holding anger in. 
It does not mean that you will never feel angry. 
Anger management is managing the behavioral responses that can arise when you are feeling angry.

All you really need to do right now is take a few moments just to relax, for you, to help you feel relaxed and calm. It feels good to relax. 
After this short relaxation session is over, you can proceed with your day, and react in a way that you choose…. relaxing for a moment now will help you to react calmly, 
rather than acting out of emotion.

It’s okay to be angry. Just allow yourself to feel however it is you are feeling right now, noticing this feeling, but not reacting just yet. 
All you’re doing is observing. 
Emotions are neither right nor wrong… they just are.

Take a deep breath in. 
Hold for a moment, and now breathe out.

Breathe in… hold that tension…. and now breathe out…. feeling the tension release with your breath.
"""


angry_6 = """
It may be surprising but humor is a great technique to talk yourself out of your anger. Anger is healthy but if you are reaching to a
point where it is damaging your relationship the advice would be to solve this matter of your emotions.
Anger portrays incorrect images of you in front of individuals in a converstation. A joke to get you laugh would
set you up for success in managing your anger.

Here is a video full of jokes to make crack up: https://youtu.be/NMIjkqZFN9o
"""

angry_7 = """
If the anger seems to be on the scale of 9-10 try to find yourself to a private room because screaming may help your temper 
to cool down as anger would be boiling inside of you. If screaming is not your jam than singing may be (no pun intended). 
Channeling those feelings into a song may help the anger be released. 

Once you are rested, check out this video for alternatives on what you can do next time if you are angry that is 
appropriate in a social situation:
https://youtu.be/BsVq5R_F6RA
"""

angry_8 = """
If do not have the guts to share your anger with a human being but know that sharing it to somewhere may help, maybe journaling is the
better alternative. Journaling has no rules, just jot your feelings down and let your hand, pencil, and your emotion mind guide 
(in this case you can listen to emotion mind only on paper). If writing is not for you, draw out your emotional state. Drawing
one's emotional state can be triggering so perhaps a change towards articulating pleasant scenery may better help.

Here is a therapeutic art activity to manage your anger:
https://youtu.be/Ptu2ca2Ezp0?t=163
"""

angry_feedbacks = [angry_2, angry_2, angry_3, angry_4, angry_5, angry_6, angry_7, angry_8, RULER]
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)

# nlp model
# intiating the model with Sequential(), Sequential() enables us to create a sequence of layers
model = Sequential()

# embedding enables the model to categorize already what textual data goes into which class and analyzes further with the following layers
# # makes a multidimensional plane where each vector will have slope based on sum, the direction of slope will determine the direction of which class dimension it goes more towards
# first param is the # of words there are to learn
# second param tells model that our output dimension is 16, each word is represented as a 16 dimensional array (substantial amount of info)
# third param tells model that the input size is 50
model.add(Embedding(10000, 16, input_shape=(50,)))

# lstm layer, bidirectional input is being able to compared in any direction (left and right) regarding the context of the sentence as lstm looks at full word with the help of rnn
# LSTM allows us to look further into the context of the sentnece as it is different than a recurrent neural network since it uses cell states to keep track of data over time periods
# returning sequences so the next layer of LSTM will know where to continue (stacking LSTM layers)
model.add(Bidirectional(LSTM(20, return_sequences=True)))

# another lstm layer
model.add(Bidirectional(LSTM(20)))

# deeply connected, dense layer to make final analyzations ~ output layer
model.add(Dense(6, activation="softmax"))

model.load_weights("nlp_model_weights.h5")

freq = 44100 # 44100 audio samples per second (utilizing sampling method to capture analog data)
duration = 5 # 5 seconds for each frame of audio

# recordings directory as a list
# * means all of the names of the files shall be joined together as one list using os.path.join
recordings_dir = os.path.join('recordings', '*')

# loading whisper model, small is the type of model (parameters vary for each model) 
asr_model = whisper.load_model("base")

# which audio files have been transcribed
transcribed = []
TRANSCRIPT_FILE = "transcript.txt"


while True:
    # time right now
    ts = datetime.datetime.now()

    # time right now is the name of the file by putting the time in string format using format codes
    filename = ts.strftime("%Y-%m-%d %H:%M:%S")
    
    # start recording and the settings are the given parameters
    # for each second there are 44100 frames therefore mutliplying it by 5 seconds (the duration) gives us a complete audio recording
    recording = sd.rec(int(duration * freq), samplerate=freq, channels=1)

    # giving the audio recording sometime
    sd.wait()

    # operating systems can not read ":" in file names
    filename = filename.replace(':', '_')

    # writing audio to the file we declared the name abiove
    # sampwidth = 2 means 4 possible amplitude values (vocab for how high or low the audio is)
    wv.write(f"recordings/{filename}.wav", recording, freq, sampwidth=2)

    

    # getting all the names of the files from recent to oldest
    # iglob searches for specific files that have a similar pattern in their name (matches our situation) it searches then sorts it (the rule above)
    files = sorted(glob.iglob(recordings_dir), key=os.path.getctime, reverse=True)
    if len(files) < 1:
        continue


    # first is the recent one as it was sorted previously (sorted list is files list)
    latest_recording = files[0]

#     latest_recording_filename = latest_recording.split('/')[1]
#     print(files[0])
    # checking if the path of the latest file exists and if it is not in the list of transcribed audio
    if os.path.exists(latest_recording) and not latest_recording in transcribed:
        # loading audio of latest recording
        audio = whisper.load_audio(latest_recording)

        # padding the audio (adding more) or trimming the audio to meet is required length of 220500 samples
        audio = whisper.pad_or_trim(audio)

        # visualizes sound waves in a different way and send it to the device the asr model is predicting on 
        # (helps the asr model out specifically for this type of problem)
        mel = whisper.log_mel_spectrogram(audio).to(asr_model.device)

        # specific instructions on how the model should behave when outputting its prediction
        # english language only and no fp16 (no fast transcription, take time)
        options = whisper.DecodingOptions(language= 'en', fp16=False)

        # transcribing using model, the distinct looking soundwaves and the format in which it should be outputted
        result = whisper.decode(asr_model, mel, options)

        # if there is less probability of no speech going on, then we want to do something
        if result.no_speech_prob < 0.5:
            # print the transcription in the console
            print(Back.WHITE, Fore.BLACK, result.text)
            
            if result.text:

                sequences = tokenizer.texts_to_sequences([result.text])
                padded = pad_sequences(sequences, truncating="post", padding="post", maxlen=50)
                
#                 print("Sequences:", sequences)
#                 print("Padded Sequences:", padded)
#                 print("Model Input Shape:", padded.shape)
                
                predictions = model.predict(padded, verbose = 2)
#                 print("Predictions:", predictions)

                # if there is some content in the prediciton list
                if predictions.any():
                    # dict with every key,value pair of emotion labels helps output the emotion visually
                    emotion_mapping = {0: "Fearful", 1: "Sadness", 2: "Love", 3: "Anger", 4: "Fear", 5: "Surprise"}
                    # maximum prediction
                    predicted_index = np.argmax(predictions)
                    
                    # retrieve correct emotion based on current index
                    predicted_emotion = emotion_mapping.get(predicted_index, "Unknown")
                    
                    # Print the predicted emotion
                    print(Back.WHITE, Fore.BLACK, "Predicted Emotion:", predicted_emotion)
                    
                    if predicted_index == 0:
                        # shuffle the feedback list to randomize
                        shuffled_fearful = sorted(fearful_feedbacks, key=lambda x: random.random())
                        
                        # get the first element of the shuffled list (will be different for each iteration)
                        random_fearful_feedback = shuffled_fearful[0]
                        print(Back.GREEN, Fore.WHITE, random_fearful_feedback, ".Do not say anything for 45 seconds as the program will not read it.")
                        time.sleep(20)
                    elif predicted_index == 1:
                        # shuffle the feedback list to randomize
                        shuffled_sad = sorted(sad_feedbacks, key=lambda x: random.random())
                        
                        # get the first element of the shuffled list (will be different for each iteration)                        
                        random_sad_feedback = shuffled_sad[0]
                        print(Back.BLUE, Fore.BLACK, random_sad_feedback, ".Do not say anything for 45 seconds as the program will not read it.")
                    elif predicted_index == 3:
                        shuffled_angry = sorted(angry_feedbacks, key=lambda x: random.random())
                        # get the first element of the shuffled list (will be different for each iteration)                        
                        random_angry_feedback = shuffled_angry[0]
                        print(Back.RED, Fore.BLACK, random_angry_feedback, ".Do not say anything for 45 seconds as the program will not read it.")

                    
#                     print("Predicted Index:", predicted_index)
                    
                    
                    
                else:
                    print("Warning: Predictions are empty.")

            else:
                print("Warning: Result.text is empty.")



#             if np.argmax(predictions) == 0:
#                 print("Sadness")
#             elif np.argmax(predictions) == 1:
#                 print("Joy")
#             elif np.argmax(predictions) == 2:
#                 print("Love")
#             elif np.argmax(predictions) == 3:
#                 print("Anger")
#             elif np.argmax(predictions) == 4:
#                 print("Fear")
#             elif np.argmax(predictions) == 5:
#                 print("Surprise")

            # writing down all transcriptions into a file using the config library, 'a' stands for appending to a file
            # append text to transcript file
            with open(TRANSCRIPT_FILE, 'a') as f:
                f.write(result.text)
        # save list of transcribed recordings so that we don't transcribe the same one again
        transcribed.append(latest_recording)


                                                text  label
0                            i didnt feel humiliated      0
1  i can go from feeling so hopeless to so damned...      0
2   im grabbing a minute to post i feel greedy wrong      3
3  i am ever feeling nostalgic about the fireplac...      2
4                               i am feeling grouchy      3


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:89: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


  I'm angry.
1/1 - 0s - 273ms/step
  Predicted Emotion: Anger
  
It may be surprising but humor is a great technique to talk yourself out of your anger. Anger is healthy but if you are reaching to a
point where it is damaging your relationship the advice would be to solve this matter of your emotions.
Anger portrays incorrect images of you in front of individuals in a converstation. A joke to get you laugh would
set you up for success in managing your anger.

Here is a video full of jokes to make crack up: https://youtu.be/NMIjkqZFN9o
 .Do not say anything for 45 seconds as the program will not read it.
  No.
1/1 - 0s - 10ms/step
  Predicted Emotion: Fearful
  
This anxiety can be cured with simplest of things. Having a routine to divert your focus may help you conquer your everday goals without that anxiety
being a hurdle. To begin, everyday things like a good night’s sleep, a wholesome meal and a walk are often the best cures for anxiety.
Even it could be having supportive people aro